In [5]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import logging as log
import sys, os
import copy
import pyomo.environ as pyo
import sys
import openpyxl
from tqdm import tqdm
import pkg_resources

%matplotlib inline

In [3]:
C_matrix_nested.shape

(20736, 127, 141)

In [6]:
# path to ODYM:
sys.path.insert(
    0,
    os.path.join(
        r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\battery_sustainability_model\ME_battery_supply_chain\Model\ODYM"
    ),
)


import ODYM_Classes as msc
import ODYM_Functions as msf

# Logfile ODYM
log_verbosity = eval("log.INFO")
log_filename = "log_file.md"
[Mylog, console_log, file_log] = msf.function_logger(log_filename, os.getcwd(), log_verbosity, log_verbosity)


# Path to the battery cost and emission scripts:
local_path = r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Bat_Sust_Model"  # ADD PATH
sys.path.insert(1, local_path)
from batt_sust_model import battery_emissions as bat_lca
from batt_sust_model import battery_cost as bat_cost


# Local path to all case study parameter files:
datapath = "."


In [8]:
# Read ODYM config file
config_file = openpyxl.load_workbook(datapath + "\model_config.xlsx", data_only=True)
script_config = {"Model Setting": config_file["Cover"]["D4"].value}
config_sheet = config_file["Setting_" + script_config["Model Setting"]]
project_name = config_sheet["D2"].value
ScriptConfig = msf.ParseModelControl(config_sheet, script_config)

# Parse configfile
(
    IT_Aspects,
    IT_Description,
    IT_Dimension,
    IT_Classification,
    IT_Selector,
    IT_IndexLetter,
    PL_Names,
    PL_Description,
    PL_Version,
    PL_IndexStructure,
    PL_IndexMatch,
    PL_IndexLayer,
    PrL_Number,
    PrL_Name,
    PrL_Comment,
    PrL_Type,
    ScriptConfig,
) = msf.ParseConfigFile(config_sheet, script_config, Mylog)

# Load classification file and establish classification dictionary:
Classfile = openpyxl.load_workbook(datapath + "\model_classification.xlsx", data_only=True)
Classsheet = Classfile["MAIN_Table"]
MasterClassification = msf.ParseClassificationFile_Main(Classsheet, Mylog)

ModelClassification = {}  # Dict of model classifications
for m in range(0, len(IT_Aspects)):
    ModelClassification[IT_Aspects[m]] = copy.deepcopy(MasterClassification[IT_Classification[m]])
    EvalString = msf.EvalItemSelectString(IT_Selector[m], len(ModelClassification[IT_Aspects[m]].Items))
    if EvalString.find(":") > -1:  # range of items is taken
        RangeStart = int(EvalString[0 : EvalString.find(":")])
        RangeStop = int(EvalString[EvalString.find(":") + 1 : :])
        ModelClassification[IT_Aspects[m]].Items = ModelClassification[IT_Aspects[m]].Items[RangeStart:RangeStop]
    elif EvalString.find("[") > -1:  # selected items are taken
        ModelClassification[IT_Aspects[m]].Items = [ModelClassification[IT_Aspects[m]].Items[i] for i in eval(EvalString)]
    elif EvalString == "all":
        None
    else:
        Mylog.info("ITEM SELECT ERROR for aspect " + IT_Aspects[m] + " were found in datafile.</br>")
        break
Mylog.info("Define index table dataframe")
IndexTable = pd.DataFrame(
    {
        "Aspect": IT_Aspects,
        "Description": IT_Description,
        "Dimension": IT_Dimension,
        "Classification": [ModelClassification[Aspect] for Aspect in IT_Aspects],
        "IndexLetter": IT_IndexLetter,
    }
)  # Unique one letter indices to be used later for calculations.

# Default indexing of IndexTable
IndexTable.set_index("Aspect", inplace=True)

# Add indexSize to IndexTable:
IndexTable["IndexSize"] = pd.Series(
    [len(IndexTable.Classification[i].Items) for i in range(0, len(IndexTable.IndexLetter))],
    index=IndexTable.index,
)

# list of the classifications used for each indexletter
IndexTable_ClassificationNames = [IndexTable.Classification[i].Name for i in range(0, len(IndexTable.IndexLetter))]

IndexTable


INFO (ODYM_Functions.py <ParseConfigFile>): Read parameter list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read process list from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model run control from model config sheet.
INFO (ODYM_Functions.py <ParseConfigFile>): Read model output control from model config sheet.
INFO (2943212141.py <<cell line: 49>>): Define index table dataframe


,Description,Dimension,Classification,IndexLetter,IndexSize
Aspect,,,,,
Regions,Battery production locations in Europe,Region,<ODYM_Classes.Classification object at 0x00000...,r,8
Process,All foreground processes,Process,<ODYM_Classes.Classification object at 0x00000...,p,141
Bat_process,Battery manufacturing process,Process,<ODYM_Classes.Classification object at 0x00000...,b,27
Goods,All foreground goods,Material,<ODYM_Classes.Classification object at 0x00000...,g,127
Internal_goods,Goods flowing inside factory,Material,<ODYM_Classes.Classification object at 0x00000...,i,15
Waste,Waste materials,Material,<ODYM_Classes.Classification object at 0x00000...,w,12
Energy,Electricity and heat,Energy,<ODYM_Classes.Classification object at 0x00000...,n,2
Elements,Chemical elements,Material,<ODYM_Classes.Classification object at 0x00000...,e,11
Emissions,ReCiPe Midpoint (H) V1.13 no LT,Satellite,<ODYM_Classes.Classification object at 0x00000...,h,17


In [9]:
# Index shortcuts:
r = IndexTable.loc["Regions", "Classification"].Items
p = IndexTable.loc["Process", "Classification"].Items
g = IndexTable.loc["Goods", "Classification"].Items
D = IndexTable.loc["Battery_designs", "Classification"].Items
P = IndexTable.loc["Technical_parameters", "Classification"].Items
bp = IndexTable.loc["Bat_process", "Classification"].Items
h = IndexTable.loc["Emissions", "Classification"].Items
f = IndexTable.loc["Factors", "Classification"].Items
e = IndexTable.loc["Elements", "Classification"].Items
p_electricity = [x for x in p if "market group for electricity battery production, medium voltage" in x]
p_gas = [x for x in p if "market group for heat, district or industrial, natural gas" in x]


In [ ]:
C_matrix_nested = np.load(r"C:\Users\Joris\Desktop\C_matrix_nested.npy")


In [ ]:

manufacturing_capacity = [20000, 500000]



factors = IndexTable.loc["Factors", "Classification"].Items
battery_production_index = [p.index(i) for i in bp]


F_physical_nested_capacity = np.zeros((2, len(D), len(factors), len(p)))
for i,cap in enumerate(manufacturing_capacity):
    for x in tqdm(D):
        param_dict_all[x]["battery_manufacturing_capacity"] = cap
        F_matrix = bat_cost.factors_battery_production(
            param_dict_all[x], run_multiple=default_cost_lst
        ).reindex(columns=bp).fillna(0) #Fill cathode binder solvent waste handling with 0
        F_physical_nested_capacity[i,x, :, battery_production_index] = F_matrix.T
manufacturing_capacity = 100000

In [10]:
pd.DataFrame(C_matrix_nested[0], g,p)

,anode active material (natural graphite) import,anode active material (SiO) import,anode active material (synthetic graphite) import,anode binder (CMC) production,anode binder additive (SBR) production,anode calendering,anode coating and drying,anode current collector Cu (10um) production,anode current collector Cu (11um) production,anode current collector Cu (12um) production,...,"market group for electricity battery production, medium voltage, PL","market group for electricity battery production, medium voltage, SE","market group for heat, district or industrial, natural gas, DE","market group for heat, district or industrial, natural gas, European average","market group for heat, district or industrial, natural gas, FR","market group for heat, district or industrial, natural gas, GB","market group for heat, district or industrial, natural gas, HU","market group for heat, district or industrial, natural gas, NO","market group for heat, district or industrial, natural gas, PL","market group for heat, district or industrial, natural gas, SE"
"aluminium, wrought alloy",0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anode active material (natural graphite),9.723422,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anode active material (SiO),0.000000,63.9936,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anode active material (synthetic graphite),0.000000,0.0000,12.959323,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
anode binder (CMC),0.000000,0.0000,0.000000,5.3328,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
waste anode slurry,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
waste cathode current collector Al,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
waste cathode slurry,0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"wastewater, average",0.000000,0.0000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
